定义CSPdarknet53基本骨架网络

In [7]:
from functools import wraps

from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Add, BatchNormalization, Concatenate,
                                     Conv2D, Layer, LeakyReLU, MaxPooling2D,
                                     UpSampling2D, ZeroPadding2D)
from tensorflow.keras.regularizers import l2

网络架构图：

<img src="https://pic3.zhimg.com/80/v2-139a50003c09efe54b2db906710f6252_1440w.jpg" alt="img" style="zoom:50%;" />

基本组件：
1. **CBM：**Yolov4网络结构中的最小组件，由Conv+Bn+Mish激活函数三者组成。
2. **CBL：**由Conv+Bn+Leaky_relu激活函数三者组成。
3. **Res unit：**借鉴Resnet网络中的残差结构，让网络可以构建的更深。
4. **CSPX：**借鉴CSPNet网络结构，由卷积层和X个Res unint模块Concate组成。


### 定义 compose 函数
使用Python的Lambda表达式，顺次执行函数列表，且前一个函数的输出是后一个函数的输入。compose函数适用于在神经网络中连接两个层。

In [13]:
def compose(*funcs):
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)
    else:
        raise ValueError('Composition of empty sequence not supported.')

### 定义 Mish 激活函数

In [15]:
class Mish(Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True
        
    def call(self, x):
        return x * K.tanh(K.softplus(x))
    
    def get_config(self):
        config = super(Mish, self).get_config()
        return config
    
    def compute_output_shape(self, input_shape):
        return input_shape

### 定义DarknetConv2D


In [16]:
@wraps(Conv2D)
def DarknetConv2D(*args, **kwargs):
    """Wrapper to set Darknet parameters for Convolution2D."""
    
    darknet_conv_kwargs = {'kernel_regularizer': l2(5e-4)}    
    darknet_conv_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
    darknet_conv_kwargs.update(kwargs)
    return Conv2D(*args, **darknet_conv_kwargs)

### 定义CBM 基本模块

In [17]:
def DarknetConv2D_BN_Mish(*args, **kwargs):
    """Darknet Convolution2D followed by BatchNormalization and Mish."""
    
    no_bias_kwargs = {'use_bias': False}  # 没懂为啥用 no_bias
    no_bias_kwargs.update(kwargs)
    return compose(
        DarknetConv2D(*args, **no_bias_kwargs),
        BatchNormalization(),
        Mish()
    )

### 定义 CSP 结构块

In [18]:
def CSP_Block(x, num_filters, num_blocks, all_narrow=True):
    # 填充x的边界为0，由(?, 416, 416, 32)转换为(?, 417, 417, 32)。
    # 因为下一步卷积操作的步长为2，所以图的边长需要是奇数。
    # 对高和宽进行压缩
    x = ZeroPadding2D(((1, 0), (1, 0)))(x)
    x = DarknetConv2D_BN_Mish(num_filters, (3,3), strides=(2,2))(x)
    
    # 残差
    x_1 = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(x)
    
    # 主干
    x_2 = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(x)
    for i in range(num_blocks):
        x_blocks = compose(
                DarknetConv2D_BN_Mish(num_filters//2, (1,1)),
                DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (3,3)))(x_2)
        x_2 = Add()([x_2, x_blocks])
    x_2 = DarknetConv2D_BN_Mish(num_filters//2 if all_narrow else num_filters, (1,1))(x_2)
    
    # 主干、残差汇合
    x = Concatenate()([x_2, x_1])
    
    return DarknetConv2D_BN_Mish(num_filters,(1,1))(x)

### 定义 CSP_darknet53 主体

In [19]:
def darknet_body(x):
    x = DarknetConv2D_BN_Mish(32, (3,3))(x)
    x = resblock_body(x, 64, 1, False)
    x = resblock_body(x, 128, 2)
    x = resblock_body(x, 256, 8)
    feat1 = x
    x = resblock_body(x, 512, 8)
    feat2 = x
    x = resblock_body(x, 1024, 4)
    feat3 = x
    return feat1,feat2,feat3